#### Simple Gen AI APP Using Langchain

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
## langchain Tracking 
os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')    
os.environ["LANGCHAIN_TRACKING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] =  os.getenv('LANGCHAIN_PROJECT')


In [3]:
## Data Ingesion.... we need to scrape data from websites
from langchain_community.document_loaders import WebBaseLoader


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
loader = WebBaseLoader("https://docs.smith.langchain.com/administration/tutorials/manage_spend")
loader

In [5]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='\n\n\n\n\nOptimize tracing spend on LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceAdministrationTutorialsOptimize tracing spend on LangSmithOn this pageOptimize tracing spend on LangSmith\nRecommended R

#### laod Data ---> Docs---> Divide Documents into chunks  Documents--->text--> vectors--->by using vectors Embedding ---> store in Vector store DB

In [6]:
### laod Data ---> Docs---> Divide our text into chunks--->text--> vectors--->vectors Embedding ---> store in Vector store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)

chunk_of_docs = text_splitter.split_documents(docs)



In [7]:
chunk_of_docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceAdministrationTutorialsOptimize tracing spend on LangSmithOn this pageOptimize tracing spend on LangSmith\nRecommended ReadingBefo

In [8]:
### embedding technique
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [8]:
from langchain_community.vectorstores import FAISS
vector_store_db = FAISS.from_documents(documents=chunk_of_docs, embedding=embeddings)

 

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-*******************************************************************************************************************************************************0NEA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [11]:
vector_store_db

NameError: name 'vector_store_db' is not defined

In [ ]:
# query from a vector store db
query = "Langsmith has two usage limits: total traces and extended"
results = vector_store_db.similarity_search(query=query)
results[0].page_content


In [ ]:
## create my llm
from langchain.llms import ChatOpenAI
llm = ChatOpenAI(model = "gpt-4o-mini")

In [ ]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain 
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provide context:
<context>
{context}
</context>

"""
)

document_chain = create_stuff_documents_chain(
    llm, prompt
)
document_chain

In [ ]:
## creating documents manualy
from langchain_core.documents import Document
document_chain.invoke({
    "input": "Langsmith has two usage limits: total traces and extended",
    "context": [Document(page_content="Langsmith has two usage limits: total traces and extended.These correspond to the two metrics we've been tracking on our usage graph. We can use these in tandem to have granular control over spend.")]

})

In [ ]:
### Retriever --->can be consider as interface.... as way to get vectoredb
vector_store_db


In [ ]:
retriever = vector_store_db.as_retriever()
from langchain.chains import create_retrieval_chain
retriever_chain = create_retrieval_chain(retriever, document_chain)




In [ ]:
## get the response from the llm
response =  retriever_chain.invoke({"input": "Langsmith has two usage limits: total traces and extended"})
response['answer']

In [ ]:
response

In [ ]:
response[context]